<a href="https://colab.research.google.com/github/rogeriomoreirajr-jornal/dataPoxa/blob/master/EmpresariosSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quem são os empresários dos jogadores dos grandes clubes catarinenses?
- Pegar os códigos dos clubes, e montar um dicionário
- Montar uma tabela com: (nome, empresario, equipe, posição, data de nascimento, na equipe desde, **outros dados**)


In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import re
from time import sleep
import unicodedata
from IPython.display import display, Markdown

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.lower().decode('utf8')

pd.set_option('display.max_colwidth', -1)

s = requests.Session()

def clip(df, **kwargs):
	return df.to_clipboard(float_format=CommaFloatFormatter(), sep='\t', **kwargs)

class CommaFloatFormatter:
	def __mod__(self, x):
		return str(x).replace('.',',')

def make_soup(url):
    html = s.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36'})
    soup = BeautifulSoup(html.content, "html5lib")
    return soup

def get_data(soup, string):
    try:
        str_ = soup.find('th', text=string).next_sibling.next_sibling.text
        str_ = re.search('\n? *(.+[^ ]) *\t*', str_).group(1)
        return str_.replace('\xa0\xa0','')

    except:
        return None

def get_jogador(link):
    soup_ = make_soup(base+link)

    id2_ = link.split('/')[-1]

    nome = get_data(soup_, 'Nome completo:')
    posi = get_data(soup_, 'Posição:')
    data = get_data(soup_, 'Data de nascimento:')
    dataEquipe = get_data(soup_, 'Na equipa desde:')
    empresario = get_data(soup_, 'Empresário:')

    try:
      outros_ = soup_.find('span','headerueberschrift').parent.parent
      outros_ = outros_.find('div','content').text
      outros =  re.search('[ \n\t]+(.+[^ \n])[ \n\t]+', outros_).group(1)

    except: outros = None
      
    return({'nome': nome,
            'posi': posi,
            'data': data, 
            'dataEquipe': dataEquipe, 
            'empresario': empresario, 
            'outros': outros})


base = 'https://www.transfermarkt.pt'

In [39]:
times = {'Avaí':2035,
        'Figueirense':4064,
        'Chapecoense':17776,
        'Criciúma':7178,
        'Joinville':3330}

urlTime = 'https://www.transfermarkt.pt/-/startseite/verein/{}'.format

l_ = []

for time in times:
  print(time)
  soupTime = make_soup(urlTime(times[time]))
  lJogadores = {el['href'] for el in soupTime.find('table','items')\
                .findAll('a', 'spielprofil_tooltip')}
  
  for jogador in lJogadores:
    print(jogador)
    jog = get_jogador(jogador)
    jog['equipe'] = time
    l_.append(jog)

Avaí
/lucas-falcao/profil/spieler/643195
/leo-lopes/profil/spieler/595125
/marcinho/profil/spieler/653343
/gege/profil/spieler/299596
/luanderson/profil/spieler/667197
/eduardo-kunde/profil/spieler/606281
/lourenco/profil/spieler/515068
/paulinho/profil/spieler/411562
/matheus-barbosa/profil/spieler/181784
/romulo/profil/spieler/349515
/brenner/profil/spieler/435046
/feliciano-brizuela/profil/spieler/650998
/dudu/profil/spieler/643196
/joao-paulo/profil/spieler/519854
/douglas/profil/spieler/34332
/jonny-mosquera/profil/spieler/132002
/getulio/profil/spieler/592345
/daniel-amorim/profil/spieler/490613
/mauricio/profil/spieler/524571
/leo-morais/profil/spieler/150085
/gustavo-alcino/profil/spieler/561457
/gustavo-ferrareis/profil/spieler/348264
/gabriel-lima/profil/spieler/553870
/vladimir/profil/spieler/98930
/julinho/profil/spieler/177545
/richard-franco/profil/spieler/511925
/igor-fernandes/profil/spieler/250841
/gledson/profil/spieler/85965
/luan-pereira/profil/spieler/527394
/caio-

In [49]:
df = pd.DataFrame(l_)
cols = set(df.columns)-{'empresario', 'outros'}
s = df.empresario.value_counts()

for empresario in s[s>1].index:
  display(Markdown('\n'*3+'### '+empresario))
  display(df[df.empresario==empresario][cols])




### Marcio Bittencourt Sports

,equipe,dataEquipe,posi,nome,data
48,Figueirense,21/01/2019,Ataque - Segundo Avançado,None,13/04/1994
50,Figueirense,01/01/2017,Médio - Médio Defensivo,Matheus Sousa Pereira,31/01/1997
60,Figueirense,08/04/2019,Médio - Médio Ofensivo,Antônio de Moura Carvalho,29/05/1986
93,Chapecoense,29/04/2019,Defesa - Lateral Direito,None,23/05/1992
117,Criciúma,01/01/2019,Médio - Médio Ofensivo,Isnairo Reis Silva Morais,06/01/1993
118,Criciúma,18/04/2019,Médio - Médio Defensivo,Nataliel Costa da Silva,22/05/1989
139,Criciúma,01/01/2018,Defesa - Defesa Central,Sandro Silva de Souza,17/05/1988





### sem agente

,equipe,dataEquipe,posi,nome,data
33,Avaí,16/08/2016,Defesa - Defesa Central,Ebert William Amâncio,11/11/1983
75,Figueirense,12/12/2018,Defesa - Defesa Central,Rafael Berger,14/07/1986
81,Chapecoense,30/07/2018,Defesa - Defesa Central,None,18/11/1984
92,Chapecoense,12/02/2015,Defesa - Defesa Central,Hélio Hermito Zampier Neto,16/08/1985
94,Chapecoense,29/05/2019,Médio - Médio Ofensivo,Fernando Camilo Farias,09/03/1986
104,Chapecoense,01/01/2019,Médio - Médio Direito,None,19/01/1990
143,Criciúma,22/03/2019,Ataque - Ponta de Lança,Leonardo Gamalho de Souza,30/01/1986





### AGN Football

...

,equipe,dataEquipe,posi,nome,data
16,Avaí,03/01/2019,Ataque - Ponta de Lança,Getulio Wandelly Silva Timóteo,10/06/1997
28,Avaí,01/06/2017,Médio - Médio Centro,Luan Pereira,21/04/2000





### Showball

,equipe,dataEquipe,posi,nome,data
121,Criciúma,-,Guarda-redes,Wagner Aparecido de Andrade Júnior,20/09/1999
141,Criciúma,01/01/2019,Guarda-redes,None,05/03/1987





### GMRsports

,equipe,dataEquipe,posi,nome,data
83,Chapecoense,05/01/2018,Médio - Médio Defensivo,None,11/06/1984
108,Chapecoense,01/01/2018,Médio - Médio Esquerdo,None,23/08/1989
